# Notebook 1; Alternative brain eigenmodes:

This notebook contains the scripts that generate several alternative brain eigenmodes that were used in our evaluations.


## Preliminary scripts

---

These scripts load required packages and define useful functions that are used by this notebook.

##### Package imports

---


In [1]:
import os
import gc
import sys
import glob
import json
import random
import datetime
import importlib
import itertools
import numpy as np
from scipy import spatial
import scipy.sparse as sparse
import scipy.stats as stats
import scipy.io as sio
import pandas as pd
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
import boto3
import lapy
import h5py
from sklearn import linear_model

# CSS used for computing local distances and connectome smoothing
from Connectome_Spatial_Smoothing import CSS as css

# Cerebro brain viewer used for visualization
from cerebro import cerebro_brain_utils as cbu
from cerebro import cerebro_brain_viewer as cbv


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


##### Basic functions

---


In [2]:
# Some useful functions

class MyNumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)


def ensure_dir(file_name):
    os.makedirs(os.path.dirname(file_name), exist_ok=True)
    return file_name


def list_dirs(path=os.getcwd()):
    files = glob.glob(os.path.join(path, '*'))
    files = [x for x in files if os.path.isdir(x)]
    return files


def file_exists(file_name, path_name=os.getcwd()):
    return os.path.isfile(os.path.join(path_name, file_name))


def write_json(json_obj, file_path):
    with open(file_path, 'w') as outfile:
        json.dump(json_obj, outfile, sort_keys=True, indent=4,
                  cls=MyNumpyEncoder)
    return json_obj


def load_json(file_path):
    with open(file_path, 'r') as infile:
        return json.load(infile)


def write_np(np_obj, file_path):
    with open(file_path, 'wb') as outfile:
        np.save(outfile, np_obj)


def load_np(file_path):
    with open(file_path, 'rb') as infile:
        return np.load(infile)


##### Directory structure

---


In [3]:
# path setting
main_dir = os.path.abspath('../')

data_dir = f"{main_dir}/data"


##### Extracting mesh connectivity structure

---


In [4]:
# basic parameters
surface = 'midthickness_MSMAll'

# load an example dscalar
dscalar_file = f'{data_dir}/templates/ones.dscalar.nii'
dscalar = nib.load(dscalar_file)

brain_models = [x for x in dscalar.header.get_index_map(1).brain_models]

# load surfaces for visualization
left_surface_file = f'{data_dir}/templates/S1200.L.{surface}.32k_fs_LR.surf.gii'
left_surface = nib.load(left_surface_file)
right_surface_file = f'{data_dir}/templates/S1200.R.{surface}.32k_fs_LR.surf.gii'
right_surface = nib.load(right_surface_file)

# create a mapping between surface and cifti vertices
left_cortical_surface_model, right_cortical_surface_model = brain_models[0], brain_models[1]
cifti_to_surface = {}
surface_to_cifti = {}
for (i, x) in enumerate(left_cortical_surface_model.vertex_indices):
    cifti_to_surface[i] = x
    surface_to_cifti[x] = i
for (i, x) in enumerate(right_cortical_surface_model.vertex_indices):
    cifti_to_surface[i + right_cortical_surface_model.index_offset] = x + right_surface.darrays[0].data.shape[0]
    surface_to_cifti[x + right_surface.darrays[0].data.shape[0]] = i + right_cortical_surface_model.index_offset

# construct data over surface
surface_mask = list(surface_to_cifti.keys())

left_surface_mask = surface_mask[:right_cortical_surface_model.index_offset]
right_surface_mask = surface_mask[right_cortical_surface_model.index_offset:]


In [ ]:
def get_surface_indices_left_and_right_on_cifti(cifti=dscalar):
    # load the brain models from the file (first two models are the left and right cortex)
    brain_models = [x for x in cifti.header.get_index_map(1).brain_models]
    # get the names of brain structures in the cifti file
    structure_names = [x.brain_structure for x in brain_models]
    
    left_cortex_index = structure_names.index('CIFTI_STRUCTURE_CORTEX_LEFT')
    right_cortex_index = structure_names.index('CIFTI_STRUCTURE_CORTEX_RIGHT')
    
    left_surfaces_cifti_indices = [x for x in brain_models[left_cortex_index].vertex_indices]
    right_surfaces_cifti_indices = [x for x in brain_models[right_cortex_index].vertex_indices]
    
    return (left_surfaces_cifti_indices, right_surfaces_cifti_indices)


In [ ]:
# let's load the cortical surfaces and a sample cifti file first
left_surf = left_surface
right_surf = right_surface

cifti = dscalar


# constructing the local connectivity for surfaces:
left_surf_dim = left_surf.darrays[0].data.shape[0]
left_surf_adj = sparse.dok_matrix((left_surf_dim, left_surf_dim), dtype=np.float32)

for t in left_surf.darrays[1].data:
    left_surf_adj[t[0], t[1]] = 1
    left_surf_adj[t[1], t[0]] = 1
    left_surf_adj[t[2], t[1]] = 1
    left_surf_adj[t[1], t[2]] = 1
    left_surf_adj[t[0], t[2]] = 1
    left_surf_adj[t[2], t[0]] = 1


right_surf_dim = right_surf.darrays[0].data.shape[0]
right_surf_adj = sparse.dok_matrix((right_surf_dim, right_surf_dim), dtype=np.float32)

for t in left_surf.darrays[1].data:
    right_surf_adj[t[0], t[1]] = 1
    right_surf_adj[t[1], t[0]] = 1
    right_surf_adj[t[2], t[1]] = 1
    right_surf_adj[t[1], t[2]] = 1
    right_surf_adj[t[0], t[2]] = 1
    right_surf_adj[t[2], t[0]] = 1

left_surf_adj = left_surf_adj.tocsr()
right_surf_adj = right_surf_adj.tocsr()

left_surfaces_cifti_indices, right_surfaces_cifti_indices = get_surface_indices_left_and_right_on_cifti()

# cortical surface local connectivity (excluding medial wall)
left_surf_adj_selection = left_surf_adj[:,left_surfaces_cifti_indices][left_surfaces_cifti_indices,:]
right_surf_adj_selection = right_surf_adj[:,right_surfaces_cifti_indices][right_surfaces_cifti_indices,:]

# aggregating two cortical surfaces
local_adjacency = sparse.block_diag((left_surf_adj_selection, right_surf_adj_selection))


In [ ]:
left_local_connections = left_surf_adj_selection.copy()
left_local_connections.eliminate_zeros()


##### Graph Laplacian scripts

---


In [ ]:
# A function to compute the symmetric normalized adjacency
def symmetric_normalized_adj(adj):
    degree = sparse.diags(np.array(adj.sum(axis=0))[0])
    inverse_degree_square_root = degree.copy()
    inverse_degree_square_root.data **= -0.5
    SNA = (inverse_degree_square_root*adj*inverse_degree_square_root)
    return SNA


# a function performing a symmetric laplacian eigenmode decomposition over a sparse adjacency structure
def laplace_spectrum_rev(adj, k=500):
    # use shift invert mode
    lambdas, vectors = sparse.linalg.eigsh(
        symmetric_normalized_adj(adj),
        k=k+1,
        which="LM",
    )

    # convert to laplacian lambdas
    lambdas = 1 - lambdas
    lambda_idx = np.argsort(lambdas)
    lambdas = lambdas[lambda_idx]
    vectors = vectors[:, lambda_idx]
    return (lambdas, vectors)


##### Connectome thresholding scripts

---


In [ ]:
# Take sparse adjacency matrix and remove edges lower than a threshold
def threshold_connectome(connectome, threshold):
    connectome_thresholded = connectome.multiply(
        connectome > threshold
    )
    connectome_thresholded.eliminate_zeros()
    return connectome_thresholded

def edge_count(connectome):
    connectome.eliminate_zeros()
    return (connectome.data.shape[0] - (connectome.diagonal() > 0).sum()) / 2

def find_density_threshold(connectome, density, precision=1e-5, max_iterations=50):
    # edge count to meet the desired density
    full_edge_count = (np.prod(connectome.shape) - connectome.shape[0]) * 0.5
    goal_edge_count = density * full_edge_count
    edge_count_tolerance = precision * full_edge_count

    # define a threshold range to search over
    search_range = (connectome.data.min() / 1.01, connectome.data.max())
    edge_count_range = (edge_count(connectome), 0)

    # exit if desired density is greater than the connectome density
    if goal_edge_count > edge_count_range[0]:
        print(
            f'Warning, connectome density ({edge_count_range[0] / full_edge_count}) '
            f'is lower than desired density ({density}).'
        )
        return search_range[0]

    # binary search with tolerance to find the threshold
    old_threshold, old_edge_count = search_range[1], edge_count_range[1]
    iterations = 0
    while iterations < max_iterations:
        iterations += 1
        new_threshold = np.mean(search_range)
        new_edge_count = edge_count(connectome > new_threshold)

        # check if threshold is good enough
        if abs(new_edge_count - goal_edge_count) < edge_count_tolerance:
            break

        elif new_edge_count > goal_edge_count:
            search_range = (new_threshold, search_range[1])
            edge_count_range = (new_edge_count, edge_count_range[1])

        else:
            search_range = (search_range[0], new_threshold)
            edge_count_range = (edge_count_range[0], new_edge_count)

        if iterations >= max_iterations:
            print(
                f'Warning, max iterations reached; goal density: {density}, '
                f'achieved density: {new_edge_count / full_edge_count}'
            )

    print(f"iterations: {iterations}, achieved density: {new_edge_count / full_edge_count}")
    return new_threshold

# Function to compute density of a high-resolution connectome
def density(adj):
    full_edge_count = (np.prod(adj.shape) - adj.shape[0]) * 0.5
    adj_edge_count = edge_count(adj)
    return (adj_edge_count / full_edge_count)

def report_density(connectome):
    print("Connectome density: {:.1f}%".format(100 * density(connectome)))

def binarized_connectome(connectome):
    return (connectome > 0).astype(float)

def compute_and_store_eigenmodes(connectome, prefix, store_path=f"{data_dir}/eigenmodes"):
    eigenvalues, eigenvectors = laplace_spectrum_rev(connectome)
    write_np(eigenvectors, ensure_dir(f'{store_path}/{prefix}_eigenmodes.npy'))
    
def connectome_strength(connectome):
    return np.array(connectome.sum(0))[0]


##### Connectome smoothing scripts

---


In [ ]:
# Group-level surfaces to compute geodesic distance
left_white_surface_file = f'{data_dir}/templates/S1200.L.white_MSMAll.32k_fs_LR.surf.gii'
right_white_surface_file = f'{data_dir}/templates/S1200.R.white_MSMAll.32k_fs_LR.surf.gii'

# CSS parameters
fwhm = 8
sigma = css._fwhm2sigma(fwhm)
epsilon = 0.1
max_smoothing_distance = css._max_smoothing_distance(sigma, epsilon, 2)

# Local geodesic distance computed on the cortical surfaces
cortical_local_geodesic_distances = css._get_cortical_local_distances(
    left_white_surface_file,
    right_white_surface_file,
    max_smoothing_distance,
    css._sample_cifti_dscalar
)

left_cortical_local_geodesic_distances = cortical_local_geodesic_distances[:right_cortical_surface_model.index_offset,:][:,:right_cortical_surface_model.index_offset]

# Combine the two distance matrices and convert to smoothing coefficients
left_smoothing_kernel = css._local_distances_to_smoothing_coefficients(
    left_cortical_local_geodesic_distances,
    sigma
)


## OSF deposited data:


---

First, downloading data from Pang et al. (2023) that was deposited to [OSF](https://osf.io/xczmp/).


In [ ]:
! mkdir "../data/pang"

Note: If you are planning to replicate our scripts, in the following command, you should replace `<Data_URL>` with a download link acquired from [here](https://osf.io/xczmp/files/osfstorage).

In [ ]:
! curl -JL -o "../data/pang/osf_data.zip" <Data_URL>

Once the data is downloaded, extract the compressed archive:

In [5]:
! ls "../data/pang"

empirical  osf_data.zip  results  template_eigenmodes


In [ ]:
! unzip "../data/pang/osf_data.zip" -d "../data/pang/"

Load the eigenmodes:

In [6]:
### Geometry eigenmodes:
pang_geometric_eigenmodes = np.genfromtxt(f"{data_dir}/pang/results/basis_geometric_midthickness-lh_evec_200.txt")

pang_geometric_eigenmodes.shape


(32492, 200)

In [7]:
### EDR eigenmodes:
with h5py.File(f"{data_dir}/pang/results/basis_connectome_EDR_midthickness-lh_evec_200.mat", 'r') as f:
    pang_edr_eigenmodes = np.array(f['eig_vec']).T

pang_edr_eigenmodes.shape
    

(32492, 200)

In [8]:
### Connectome eigenmodes:
with h5py.File(f"{data_dir}/pang/results/basis_connectome_midthickness-lh_evec_200.mat", 'r') as f:
    pang_connectome_eigenmodes = np.array(f['eig_vec']).T

pang_connectome_eigenmodes.shape
    

(32492, 200)

Store the eigenmodes in a NumPy binary file:

In [9]:
# number of modes
N_modes = 200


In [10]:
write_np(
    pang_geometric_eigenmodes[left_surface_mask, :N_modes],
    ensure_dir(f"{data_dir}/eigenmodes/pang_geometric_eigenmodes.npy")
)


In [11]:
write_np(
    pang_edr_eigenmodes[left_surface_mask, :N_modes],
    ensure_dir(f"{data_dir}/eigenmodes/pang_edr_eigenmodes.npy")
)


In [12]:
write_np(
    pang_connectome_eigenmodes[left_surface_mask, :N_modes],
    ensure_dir(f"{data_dir}/eigenmodes/pang_connectome_eigenmodes.npy")
)


In addition to the provided eigenmodes, we will also load and store the group-level structural connectome provided by Pang and colleagues. This is used by supplemental analytical evaluations present in our commentary. 

In [13]:
pang_connectome = sparse.csr_matrix(
    sio.loadmat(
        f"{data_dir}/pang/empirical/S255_high-resolution_group_average_connectome_cortex_nomedial-lh.mat"
    )['avgSC_L']
)
pang_connectome.shape


(29696, 29696)

In [14]:
sparse.save_npz(
    ensure_dir(f"{data_dir}/connectomes/pang_group_average_connectome.npz"),
    pang_connectome
)


## Our group-level connectome:


---

Using a recent tractography pipeline adhering to state-of-the-art connectome reconstruction decisions (detailed [here](https://doi.org/10.1016/j.neuroimage.2023.120407)), we mapped a similar group-level connectome from the identical set of HCP participants used by Pang and colleagues. The following cells loads our group-level connectome.

**Note:** Please reach out to us in case you are keen to use these connectomes in your future research.


In [15]:
our_connectome = sparse.load_npz(f"{data_dir}/connectomes/our_group_average_connectome.npz")
our_connectome.shape

(29696, 29696)

## Our connectome eigenmodes

---

In the main text of the analysis, we use connectome eigenmodes from a smoothed weighted graph constructed from the group-average connectome (loaded in the previous cell). These connectome eigenmodes were generated using the following script and are provided as part of this repository.


In [ ]:
# perform connectome-spatial smoothing
smoothed_connectome = threshold_connectome(
    css.smooth_high_resolution_connectome(
        our_connectome,
        left_smoothing_kernel
    ), 1e-4
)

# 10% density
threshold = find_density_threshold(smoothed_connectome, 0.1)
smoothed_connectome = threshold_connectome(smoothed_connectome, threshold)

# combine with local connections
smoothed_connectome = smoothed_connectome + 1e-6 * left_local_connections

# generate Laplacian eigenmodes
compute_and_store_eigenmodes(smoothed_connectome, prefix="our_connectome")


### Supplementary connectome eigenmodes:

---

In addition to the connectome eigenmodes provided above, we performed supplementary evaluations to assess the sensitivity of the findings to different connectome reconstruction steps. The following cells contain the scripts that were used to generate the eigenmodes for our supplementary evaluations.


#### Density increase:

---

We first evaluated the effect of increasing binary connectome density from 0.1% to 1%:


In [ ]:
# 1% density
threshold = find_density_threshold(pang_connectome, 0.01)
connectome = threshold_connectome(pang_connectome, threshold)

# combine with local connections
connectome = connectome + left_local_connections

# binarize
connectome = binarized_connectome(connectome)

# generate Laplacian eigenmodes
compute_and_store_eigenmodes(connectome, prefix="1%_density_binary_connectome")


#### Gyral bias mitigation:

---

Considering the visually evident presence of the gyral bias in Pang et al.'s connectome eigenmodes, we implemented two alternative approaches both of which were to alleviate the gyral bias prior to eigenmode construction. The following cells provide the scripts that produce the respective eigenmodes of these approaches. Notably, in both cases, the only variation is the gyral bias mitigation and the rest of eigenmode construction decisions are kept identical (i.e. 1% density, binary connectomes)


##### Gyral bias: regression

---


In [ ]:
# A function to linearly regress effect of curvature from strength
def regress_gyral_bias(connectome):
    # load curvature information
    curvature = nib.load(f"{data_dir}/templates/S1200.curvature_MSMAll.32k_fs_LR.dscalar.nii").get_fdata()[0]

    # get left hemisphere curvature
    node_dimension = connectome.shape[0]
    curvature = curvature[:node_dimension].reshape(-1, 1)

    # compute node strengths and degrees
    node_strengths = np.array(connectome.sum(0)).reshape(-1, 1)
    node_degrees = np.array((connectome > 0).sum(0)).reshape(-1, 1)

    # fit a linear regressions
    gyral_bias_predictions = linear_model.LinearRegression().fit(X=curvature, y=node_strengths).predict(curvature)

    # remove the gyral bias effect from connectome
    connectome_curvature_adjusted = connectome.tocoo()
    indices = connectome_curvature_adjusted.row
    connectome_curvature_adjusted.data -= 2 * (gyral_bias_predictions[indices, 0] / node_degrees[indices, 0])

    # add constant to ensure connection weights are not negative
    connectome_curvature_adjusted.data += 2 * (gyral_bias_predictions.max() / node_degrees[indices, 0])
    connectome_curvature_adjusted = connectome_curvature_adjusted.tocsr()

    # symmetric adjustment (to account for bias at both ends)
    connectome_curvature_adjusted = (connectome_curvature_adjusted + connectome_curvature_adjusted.T)/2

    # return adjusted connectome
    return connectome_curvature_adjusted


In [ ]:
# regress gyral bias
connectome = regress_gyral_bias(pang_connectome)

# 1% density
threshold = find_density_threshold(connectome, 0.01)
connectome = threshold_connectome(connectome, threshold)

# combine with local connections
connectome = connectome + left_local_connections

# binarize
connectome = binarized_connectome(connectome)

# generate Laplacian eigenmodes
compute_and_store_eigenmodes(connectome, prefix="1%_density_binary_gyral_bias_regression_connectome")


##### Gyral bias: tractography

---


In [ ]:
# gyral bias mitigation in tractography 
connectome = our_connectome

# 1% density
threshold = find_density_threshold(connectome, 0.01)
connectome = threshold_connectome(connectome, threshold)

# combine with local connections
connectome = connectome + left_local_connections

# binarize
connectome = binarized_connectome(connectome)

# generate Laplacian eigenmodes
compute_and_store_eigenmodes(connectome, prefix="1%_density_binary_gyral_bias_tractography_connectome")


#### Systematic evaluations:

---

In addition to the cases described above, we conducted a systematic evaluation of the influence of several connectome reconstruction decisions and parameters on reconstruction accuracy. These eigenmodes were constructed using automated scripts that were executed as parallel jobs on a high-performance computing platform (via slurm). The scripts used for this systematic evaluation can be found in the `scripts` folder of the repository.
